<a href="https://colab.research.google.com/github/emilstahl97/Scalable-Machine-Learning-and-Deep-Learning-ID2223/blob/dev/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
 import os

!pip install pyspark
!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow_text

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()


     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 60.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8dc218affcbce12fbe7824f1eeb54e5a172c027a100bfbc22ebd53d27af3a920
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 4.9 MB 4.3 MB/s 


In [3]:
# Import the necessary libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import functools
from sklearn.model_selection import train_test_split


****Mount notebook in your Google Drive****

In [20]:
# README - Execute this cell to mount the notebook in your google drive. 
# Exxdecute the cell and follow the link to sign and, paste the given key in the little text box. The credentials are only available for you. 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

os.mkdir("/content/drive/MyDrive/stsbenchmark")
os.chdir("/content/drive/MyDrive/stsbenchmark")
!git clone https://github.com/emilstahl97/stsbenchmark.git
!git pull


Mounted at /content/drive
Cloning into 'stsbenchmark'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [21]:
schema = StructType([
    StructField("genre", StringType(), True),
    StructField("filename", StringType(), True),
    StructField("year", StringType(), True),
    StructField("year_id", IntegerType(), True),
    StructField("score", FloatType(), True),
    StructField("sentence1", StringType(), True),
    StructField("sentence2", StringType(), True)])

# get the datasets from drive
train = spark.read.csv("/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-train.csv", sep ='\t', header=False, schema=schema)
test = spark.read.csv("/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-test.csv", sep ='\t', header=False, schema=schema)
dev = spark.read.csv("/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-dev.csv", sep ='\t', header=False, schema=schema)


train.show()
test.show()
dev.show()

+-------------+--------+--------+-------+-----+--------------------+--------------------+
|        genre|filename|    year|year_id|score|           sentence1|           sentence2|
+-------------+--------+--------+-------+-----+--------------------+--------------------+
|main-captions|  MSRvid|2012test|      1|  5.0|A plane is taking...|An air plane is t...|
|main-captions|  MSRvid|2012test|      4|  3.8|A man is playing ...|A man is playing ...|
|main-captions|  MSRvid|2012test|      5|  3.8|A man is spreadin...|A man is spreadin...|
|main-captions|  MSRvid|2012test|      6|  2.6|Three men are pla...|Two men are playi...|
|main-captions|  MSRvid|2012test|      9| 4.25|A man is playing ...|A man seated is p...|
|main-captions|  MSRvid|2012test|     11| 4.25|Some men are figh...|Two men are fight...|
|main-captions|  MSRvid|2012test|     12|  0.5|   A man is smoking.|   A man is skating.|
|main-captions|  MSRvid|2012test|     13|  1.6|The man is playin...|The man is playin...|
|main-capt

## **Normalize score column of each dataset**

In [12]:
train = train.withColumn("score", col("score")/2.5-1)
test = test.withColumn("score", col("score")/2.5-1)
dev = dev.withColumn("score", col("score")/2.5-1)

dev.select("score").describe().show()

+-------+--------------------+
|summary|               score|
+-------+--------------------+
|  count|                1500|
|   mean|-0.05443697837591158|
| stddev|  0.6001942581590352|
|    min|                -1.0|
|    max|                 1.0|
+-------+--------------------+



In [27]:
text_test = ['this is such an amazing movie!']

tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

text_preprocessed = preprocessing_layer(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [28]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

bert_results = encoder(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"][0].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.7626293   0.9928098  -0.18611859  0.36673865  0.15233742  0.6550452
  0.9681154  -0.94862705  0.00216148 -0.9877732   0.0684269  -0.9763061 ]
Sequence Outputs Shape:(128, 512)
Sequence Outputs Values:[[-0.28946307  0.34321287  0.33231482 ...  0.21300784  0.7102064
  -0.05771124]
 [-0.28742093  0.3198104  -0.2301862  ...  0.58455014 -0.21329764
   0.7269209 ]
 [-0.6615701   0.6887682  -0.8743294  ...  0.10877228 -0.26173192
   0.4785537 ]
 ...
 [-0.22561091 -0.28925657 -0.07064448 ...  0.47566032  0.8327713
   0.40025362]
 [-0.298242   -0.27473173 -0.05450522 ...  0.48849788  1.0955355
   0.18163413]
 [-0.44378197  0.00930735  0.07223716 ...  0.172901    1.1833245
   0.07898018]]


**Input Layer**

In [ ]:
text_input_u = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_u')
text_input_v = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_v')

**Preprocessing Layer**

In [ ]:
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing_u')
encoder_input_u = preprocessing_layer(text_input_u)
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing_v')
encoder_input_v = preprocessing_layer(text_input_v)

**BERT encoder layer**

In [ ]:
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder_u')
outputs_u = encoder(encoder_input_u)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder_v')
outputs_v = encoder(encoder_input_v)

**Pooled Output**

In [ ]:
avg_pool_2d  = tf.keras.layers.AveragePooling2D(pool_size=(128, 1), strides=(128, 1), name='pooling_u')
sequence_output = outputs_u["sequence_output"][0]
sequence_output = tf.reshape(sequence_output, [1, 128, 512, 1])
pooled_u = avg_pool_2d(sequence_output)[0][0]

avg_pool_2d  = tf.keras.layers.AveragePooling2D(pool_size=(128, 1), strides=(128, 1), name='pooling_v')
sequence_output = outputs_v["sequence_output"][0]
sequence_output = tf.reshape(sequence_output, [1, 128, 512, 1])
pooled_v = avg_pool_2d(sequence_output)[0][0]

**Output Layer**

In [ ]:
print(pooled_u.shape)
cos_distance = tf.keras.layers.Dot(axes = 1, normalize=True)([tf.reshape(pooled_u, [1, 512, 1]), tf.reshape(pooled_v, [1, 512, 1])]) # where cos (x)
print(cos_distance)
cos_distance = tf.keras.layers.Reshape((1,))(cos_distance)
final = tf.keras.layers.Lambda(lambda x: 1-x)(cos_distance)

#print(output_concat[:,1].shape)

#cosine_loss = tf.keras.losses.CosineSimilarity(axis=-1)
#final = cosine_loss(output_concat[:,0], output_concat[:,1])

print(final.shape)

**Model**

In [ ]:
model = tf.keras.Model(inputs=[text_input_u, text_input_v], outputs=[final])

tf.keras.utils.plot_model(model)
print(model)

model.summary()

# https://www.sbert.net/docs/training/overview.html

**Compiling Model**

In [ ]:
optimizer = tf.optimizers.Adam(0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

df = test.select("sentence1","sentence2","score").toPandas()

print(df.isna().sum())
df = df.fillna('')

#import numpy as np
#df["score"] = np.asarray(df["score"]).astype(np.float32)
#print(list(df["score"]))

results = model.evaluate([df["sentence1"],df["sentence2"]],df["score"])
print("Results: ", results)

In [ ]:
print([df["sentence1"][1],df["sentence1"][1]])
print(df["score"][1])
print(model.predict([df["sentence1"][1:2],df["sentence1"][1:2]]))


**Train**

In [ ]:
train.show(5)
model.fit()

**Prediction**

In [ ]:
classifications = model.predict()

**Evaluation**

In [ ]:
model.evaluate()